## Purpose 

This script creates a model for predicting water temperature in Shadow Mountain Reservoir using the t2023 data. This pass is a middle ground between the aggressive feature reduction and all features. For this model, we're using the `leaky_basic_5` setting in `settings.py`, which reduces the number of hidden layers.

### Import Modules

In [12]:
#high level modules
import os
import imp
import pandas as pd
import numpy as np

# ml/ai modules
import tensorflow as tf

### Custom Modules

In [13]:
# import custom modules
this_dir = "/Users/steeleb/Documents/GitHub/NASA-NW/modeling/SMR_temp_forecast/arNN_summary/"
imp.load_source("settings",os.path.join(this_dir,"settings.py"))
from settings import settings
imp.load_source("architecture", os.path.join(this_dir, "architecture.py"))
from architecture import build_model, compile_model
imp.load_source("universals", os.path.join(this_dir, "universal_functions.py"))
from universals import save_to_pickle, twotemp_labels_features

# point to data directory
data_dir = "/Users/steeleb/Documents/GitHub/NASA-NW/data/NN_train_val_test/SMR_forecast/met_summary/"

### Import train/val sets

Import and format training and validation arrays for use in model training

In [14]:
all_files = pd.Series(os.listdir(data_dir))
t2023 = all_files[all_files.str.contains('t2023')]
t2023_val = t2023[t2023.str.contains('validation')]
t2023_train = t2023[t2023.str.contains('training')]

# these files end up in no particular order, so we need to sort them
t2023_val = t2023_val.sort_values()
t2023_train = t2023_train.sort_values()

t2023_val, t2023_train



(24    validation_summary_t2023_2014_summary_v2024-11...
 38    validation_summary_t2023_2015_summary_v2024-11...
 33    validation_summary_t2023_2016_summary_v2024-11...
 2     validation_summary_t2023_2017_summary_v2024-11...
 3     validation_summary_t2023_2018_summary_v2024-11...
 31    validation_summary_t2023_2019_summary_v2024-11...
 44    validation_summary_t2023_2020_summary_v2024-11...
 17    validation_summary_t2023_2021_summary_v2024-11...
 8     validation_summary_t2023_2022_summary_v2024-11...
 dtype: object,
 26    training_summary_t2023_2014_summary_v2024-11-2...
 36    training_summary_t2023_2015_summary_v2024-11-2...
 30    training_summary_t2023_2016_summary_v2024-11-2...
 5     training_summary_t2023_2017_summary_v2024-11-2...
 1     training_summary_t2023_2018_summary_v2024-11-2...
 35    training_summary_t2023_2019_summary_v2024-11-2...
 40    training_summary_t2023_2020_summary_v2024-11-2...
 19    training_summary_t2023_2021_summary_v2024-11-2...
 11    training

In [15]:
def load_data(file):
    return pd.read_csv(os.path.join(data_dir, file), sep=',')

val1 = load_data(t2023_val.values[0])
train1 = load_data(t2023_train.values[0])

val2 = load_data(t2023_val.values[1])
train2 = load_data(t2023_train.values[1])

val3 = load_data(t2023_val.values[2])
train3 = load_data(t2023_train.values[2])

val4 = load_data(t2023_val.values[3])
train4 = load_data(t2023_train.values[3])

val5 = load_data(t2023_val.values[4])
train5 = load_data(t2023_train.values[4])

val6 = load_data(t2023_val.values[5])
train6 = load_data(t2023_train.values[5])

val7 = load_data(t2023_val.values[6])
train7 = load_data(t2023_train.values[6])

val8 = load_data(t2023_val.values[7])
train8 = load_data(t2023_train.values[7])

val9 = load_data(t2023_val.values[8])
train9 = load_data(t2023_train.values[8])

Using the function twotemp_labels_features, we can create ML-ready features and labels for the training and validation sets.

In [16]:
features1, labels_1, val_features1, val_labels_1 = twotemp_labels_features(train1, val1)
features2, labels_2, val_features2, val_labels_2 = twotemp_labels_features(train2, val2)
features3, labels_3, val_features3, val_labels_3 = twotemp_labels_features(train3, val3)
features4, labels_4, val_features4, val_labels_4 = twotemp_labels_features(train4, val4)
features5, labels_5, val_features5, val_labels_5 = twotemp_labels_features(train5, val5)
features6, labels_6, val_features6, val_labels_6 = twotemp_labels_features(train6, val6)
features7, labels_7, val_features7, val_labels_7 = twotemp_labels_features(train7, val7)
features8, labels_8, val_features8, val_labels_8 = twotemp_labels_features(train8, val8)
features9, labels_9, val_features9, val_labels_9 = twotemp_labels_features(train9, val9)

In [17]:
features1.shape, labels_1.shape, val_features1.shape, val_labels_1.shape

((672, 90), (672, 2), (85, 90), (85, 2))

### Compile and train models

Here, we'll use intentionally overfit settings to create an overfit model. This particular instance uses 2 layers containint 20 nodes each. We've increased the batch size to 64 from the previous iteration, and kept the patience at 200 and the drop out at 0.1.

In [18]:
tf.keras.backend.clear_session()
tf.keras.utils.set_random_seed(settings["leaky_basic_5"]["random_seed"])

# define the early stopping callback
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
  monitor="val_loss", 
  patience=settings["leaky_basic_5"]["patience"], 
  restore_best_weights=True, 
  mode="auto"
)

## TS cross 1
model_1 = build_model(
  features1, 
  labels_1, 
  settings["leaky_basic_5"])

model_1 = compile_model(
  model_1, 
  settings["leaky_basic_5"])

# train the model via model.fit
history_1 = model_1.fit(
  features1, 
  labels_1, 
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features1, val_labels_1],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 2
model_2 = build_model(
  features2,
  labels_2, 
  settings["leaky_basic_5"])
model_2 = compile_model(model_2, settings["leaky_basic_5"])

# train the model via model.fit
history_2 = model_2.fit(
  features2,
  labels_2,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features2, val_labels_2],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 3

model_3 = build_model(
  features3,
  labels_3,
  settings["leaky_basic_5"])
model_3 = compile_model(model_3, settings["leaky_basic_5"])

# train the model via model.fit
history_3 = model_3.fit(
  features3,
  labels_3,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features3, val_labels_3],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 4

model_4 = build_model(
  features4,
  labels_4,
  settings["leaky_basic_5"])
model_4 = compile_model(model_4, settings["leaky_basic_5"])

# train the model via model.fit
history_4 = model_4.fit(
  features4,
  labels_4,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features4, val_labels_4],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 5

model_5 = build_model(
  features5,
  labels_5,
  settings["leaky_basic_5"])
model_5 = compile_model(model_5, settings["leaky_basic_5"])

# train the model via model.fit
history_5 = model_5.fit(
  features5,
  labels_5,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features5, val_labels_5],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 6

model_6 = build_model(
  features6,
  labels_6,
  settings["leaky_basic_5"])
model_6 = compile_model(model_6, settings["leaky_basic_5"])

# train the model via model.fit
history_6 = model_6.fit(
  features6,
  labels_6,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features6, val_labels_6],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 7

model_7 = build_model(
  features7,
  labels_7,
  settings["leaky_basic_5"])
model_7 = compile_model(model_7, settings["leaky_basic_5"])

# train the model via model.fit
history_7 = model_7.fit(
  features7,
  labels_7,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features7, val_labels_7],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 8

model_8 = build_model(
  features8,
  labels_8,
  settings["leaky_basic_5"])
model_8 = compile_model(model_8, settings["leaky_basic_5"])

# train the model via model.fit
history_8 = model_8.fit(
  features8,
  labels_8,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features8, val_labels_8],
  callbacks=[early_stopping_callback],
  verbose=1,
)

## TS cross 9

model_9 = build_model(
  features9,
  labels_9,
  settings["leaky_basic_5"])
model_9 = compile_model(model_9, settings["leaky_basic_5"])

# train the model via model.fit
history_9 = model_9.fit(
  features9,
  labels_9,
  epochs=settings["leaky_basic_5"]["max_epochs"],
  batch_size=settings["leaky_basic_5"]["batch_size"],
  shuffle=True,
  validation_data=[val_features9, val_labels_9],
  callbacks=[early_stopping_callback],
  verbose=1,
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 dropout (Dropout)           (None, 90)                0         
                                                                 
 dense (Dense)               (None, 10)                910       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 1,042
Trainable params: 1,042
Non-trainable params: 0
_________________________________________________________________
___________________________________________________________

And save the models and training history, to do this, you'll need to create the following directory path:

data/NN_train_val_test/SMR_forecast/models/leaky_basic_5/

In [19]:
dump_dir = "/Users/steeleb/Documents/GitHub/NASA-NW/data/NN_train_val_test/SMR_forecast/models/leaky_5_summary_t2023/"
!mkdir -p {dump_dir}

# save models to pickle
models = [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8, model_9]

for model, i in zip(models, range(1,10)):
    save_to_pickle(model, f"{dump_dir}/model_{i}.pkl")

# save history to pickles
histories = [history_1, history_2, history_3, history_4, history_5, history_6, history_7, history_8, history_9]

for history, i in zip(histories, range(1,10)):
    save_to_pickle(history, f"{dump_dir}/history_{i}.pkl")


INFO:tensorflow:Assets written to: ram://490d0799-1868-472e-85c2-7d842c8cb3ab/assets
INFO:tensorflow:Assets written to: ram://f8052b4e-9498-40d5-bff8-2987d755b149/assets
INFO:tensorflow:Assets written to: ram://b64e1db2-b273-47bc-a353-6ab5104f644d/assets
INFO:tensorflow:Assets written to: ram://91e96841-2260-4094-8907-ae8132710fc6/assets
INFO:tensorflow:Assets written to: ram://75151fb3-af7f-409b-a8d6-50cf20bac53c/assets
INFO:tensorflow:Assets written to: ram://4ac8595c-920d-4e49-b595-25359c76d8b9/assets
INFO:tensorflow:Assets written to: ram://f92410d0-58fd-466c-8a6c-e2cae594b533/assets
INFO:tensorflow:Assets written to: ram://cc582cb3-2db0-410a-8774-e6b1683d16e9/assets
INFO:tensorflow:Assets written to: ram://9f3c7bb2-5f16-46fc-ad4d-92ae897eece2/assets
INFO:tensorflow:Assets written to: ram://0ba9770e-6e6d-4e0c-981e-0165c296b623/assets
INFO:tensorflow:Assets written to: ram://643faa20-e2eb-4d16-84f2-6d9f605d6a9b/assets
INFO:tensorflow:Assets written to: ram://3fab8e06-bec1-49b5-a3d5-